# PIGPVAE cooling R4 model

In [1]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gpytorch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import copy
from torch.utils.data import Dataset, DataLoader,random_split, Subset
from tqdm import tqdm

sys.path.append(os.path.abspath('..'))
from physics import NewtonsLaw
from GP import *
from VAE_utils import AnnealingStrategy
from metrics import *

from Models.PIGPVAE_lognorm import PIGPVAE

import warnings
# Suppress only the specific GPInputWarning coming from the exact_gp.py file
warnings.filterwarnings(
    "ignore", 
    message="The input matches the stored training data. Did you forget to call model.train()?", 
    module=".*exact_gp"
)


In [2]:
# Import RICO data 
file_path = '../Data/RICO4_Dataset_processed.hdf'
df = pd.read_hdf(file_path)
#  define surrounding temperature
df.loc[:, 'sur_temp'] = (df['RTD420'] + df['RTD417'] + df['B.ASTRHT2.T'])/3 
df_h = df.copy()
df_h = df[df['temp_change_class'] == 'h']
def select_points(group):
    return group.iloc[::10]  # Select every 10th row
df_h = df_h.groupby('Scheduler Step').apply(select_points).reset_index(drop=True)
df_h = df_h.dropna()

/var/folders/46/30710m094qv2_dhfs08nzzd00000gn/T/ipykernel_16544/3580172856.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_h = df_h.groupby('Scheduler Step').apply(select_points).reset_index(drop=True)


In [3]:
class RICO4Dataset(Dataset):
    def __init__(self, df, select_var, exclude_samples=None):
        """
        Args:
            df: DataFrame containing the data.
            select_var: List of variables to include in the dataset.
            exclude_samples: List of samples (intervals) to exclude from the dataset.
        """
        self.df = df
        self.select_var = select_var
        if exclude_samples:
            self.unique_batches = [batch for batch in df['interval'].unique() if batch not in exclude_samples]
        else:
            self.unique_batches = df['interval'].unique()

    def __len__(self):
        return len(self.unique_batches)

    def __getitem__(self, idx):
        batch = self.unique_batches[idx]
        temp = self.df[self.df['interval'] == batch][self.select_var]
        
        # Normalize and convert to tensors
        ti = torch.tensor(temp['time_within_interval'].to_numpy() / 240, dtype=torch.float32)
        Ts = torch.tensor(temp['sur_temp'].to_numpy(), dtype=torch.float32)
        heat_obs = torch.tensor(temp['B.RTD1'].to_numpy(), dtype=torch.float32)
        
        # Stack the tensors to create the input batch
        x_batch = torch.stack((heat_obs, Ts, ti), dim=1)
        
        return x_batch

# Initialize the dataset, excluding samples 53 and 5
exclude_samples = [53, 5]
select_var = ['time_within_interval', 'sur_temp', 'B.RTD1']
dataset = RICO4Dataset(df_h, select_var, exclude_samples=exclude_samples)

# Validate the updated dataset
print(f"Dataset length after exclusion: {len(dataset)}")


# Define split ratios
train_ratio = 0.7
test_ratio = 1-train_ratio

# Calculate split sizes
total_size = len(dataset)
train_size = int(train_ratio * total_size)
test_size = total_size - train_size

seed = 0
generator = torch.Generator().manual_seed(seed)

# Split the dataset
train_subset, test_subset = random_split(
    dataset,
    [train_size, test_size],
    generator=generator
)
train_loader = DataLoader(train_subset, batch_size=1, shuffle=False)
val_loader = DataLoader(test_subset, batch_size=1, shuffle=False)

Dataset length after exclusion: 29


In [4]:
# Define the range of initial_alpha values for ablation
initial_alpha_values = [0, 0.1, 0.5, 2, 100]
results = {}

# Ensure the directory exists
os.makedirs("pre_trained_models", exist_ok=True)

def train_model(initial_alpha):
    kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5))
    GP_inf = GP_inference(kernel)
    
    torch.manual_seed(1)
    model = PIGPVAE(GP_inf=GP_inf, 
                    hidden_layers_encoder_delta=[10,10], 
                    hidden_layers_encoder_phy=[10], 
                    hidden_layers_decoder_delta=[10,10],
                    activation_decoder_delta=nn.ELU(),
                    initial_alpha=initial_alpha,
                    trainable_alpha=False,
                    mu_prior=torch.tensor(1.7), var_prior=torch.tensor(1)
                   )
    
    gp_params = []
    base_params = []
    
    for name, param in model.named_parameters():
        if 'GP' in name:
            gp_params.append(param)  # Gaussian Process parameters
        else:
            base_params.append(param)  # All other parameters
    
    # Define learning rates
    base_lr = 0.001
    gp_lr = 0.0001
    
    opt = torch.optim.Adam([
        {'params': base_params, 'lr': base_lr},
        {'params': gp_params, 'lr': gp_lr}
    ])
    opt.zero_grad()
    
    t_uni = torch.tensor([
        1.,  11.,  21.,  31.,  41.,  51.,  61.,  71.,  81.,  91., 101., 111.,
        121., 131., 141., 151., 161., 171., 181., 191., 201., 211., 221., 231.
    ])
    
    num_epochs = 600
    annealing_strategy = AnnealingStrategy(strategy='linear', 
                                           total_epochs=num_epochs, 
                                           beta_min=0.01, 
                                           beta_max=0.01)
    beta_annealed = annealing_strategy.get_beta_vector()
    beta_delta = beta_annealed
    beta_phy = beta_annealed
    
    ns = total_size
    min_T, max_T = 12, 25
    T0 = min_T + (max_T - min_T) * torch.rand(ns)
    difference = 1 + (20 - 1) * torch.rand(ns)
    Ts = T0 + difference
    unique_intervals = df_h['interval'].unique()
    num_unique_intervals = len(unique_intervals)
    
    fit_res = model.fit(
        train_loader=train_loader, 
        val_loader=val_loader, 
        opt=opt, 
        beta_delta=beta_delta,
        beta_phy=beta_phy,
        num_epochs=num_epochs,
        seed=0, plot=False, df=df_h, ns=ns, T0=T0, Ts=Ts, t_uni=t_uni/240, axs_ylim=(10, 35))
    
    # Save the trained model
    model_path = f'pre_trained_models/ablation/PIGPVAE_heating_alpha_{initial_alpha}.pth'
    torch.save(model.state_dict(), model_path)
    print(f'Model saved at {model_path}')
    
    return fit_res

# Run ablation study
for alpha in initial_alpha_values:
    print(f'Training model with initial_alpha={alpha}')
    fit_results = train_model(alpha)
    results[alpha] = fit_results

Training model with initial_alpha=0


Training Progress:   5%|▌         | 31/600 [00:03<00:58,  9.65it/s]

Epoch 30: Train Loss = 2.2499, KL_phy = 2.5454, KL_delta = 0.0403, MSE = 2.2240, CE = 1.0490, Reg_Loss = 0.1286, Alpha = 0.0000
Validation Loss = 3.0433, Val_KL_phy = 2.5288, Val_KL_delta = 0.0680, Val_MSE = 3.0174, Val_CE = 1.0535, Val_Reg_Loss = 0.1846, Val_Alpha = 0.0000


Training Progress:  10%|█         | 61/600 [00:06<00:55,  9.65it/s]

Epoch 60: Train Loss = 2.0427, KL_phy = 2.3537, KL_delta = 1.6412, MSE = 2.0027, CE = 0.4653, Reg_Loss = 0.2372, Alpha = 0.0000
Validation Loss = 2.6812, Val_KL_phy = 2.3422, Val_KL_delta = 1.7200, Val_MSE = 2.6406, Val_CE = 0.4527, Val_Reg_Loss = 0.3444, Val_Alpha = 0.0000


...

Epoch 540: Train Loss = 2.1520, KL_phy = 3.0710, KL_delta = -0.4225, MSE = 2.1122, CE = 1.2205, Reg_Loss = 0.0001, Alpha = 100.0000
Validation Loss = 3.3967, Val_KL_phy = 3.0548, Val_KL_delta = -0.4245, Val_MSE = 3.3601, Val_CE = 1.2374, Val_Reg_Loss = 0.0001, Val_Alpha = 100.0000


Training Progress:  95%|█████████▌| 571/600 [01:01<00:03,  9.28it/s]

Epoch 570: Train Loss = 2.1870, KL_phy = 2.9883, KL_delta = -0.4349, MSE = 2.1483, CE = 1.3636, Reg_Loss = 0.0001, Alpha = 100.0000
Validation Loss = 3.4421, Val_KL_phy = 2.9798, Val_KL_delta = -0.4343, Val_MSE = 3.4056, Val_CE = 1.3630, Val_Reg_Loss = 0.0001, Val_Alpha = 100.0000


Training Progress: 100%|██████████| 600/600 [01:04<00:00,  9.25it/s]

Epoch 600: Train Loss = 2.0578, KL_phy = 3.0057, KL_delta = -0.4422, MSE = 2.0181, CE = 1.3683, Reg_Loss = 0.0001, Alpha = 100.0000
Validation Loss = 3.3483, Val_KL_phy = 2.9797, Val_KL_delta = -0.4417, Val_MSE = 3.3126, Val_CE = 1.3644, Val_Reg_Loss = 0.0001, Val_Alpha = 100.0000
Model saved at pre_trained_models/ablation/PIGPVAE_heating_alpha_100.pth


In [5]:
full_data = DataLoader(dataset)
original_data_list = []
for batch in full_data:
    # Each batch is expected to be of shape [24, 1]
    # Add to the list
    original_data_list.append(batch[0][:,0])

# Stack to create a tensor of shape [total_samples, 24, 1]
original_data = torch.stack(original_data_list)
original_data = original_data.unsqueeze(-1)

In [6]:
metrics = SyntheticDataMetrics()
ns = total_size
initial_alpha_values = [0, 0.1, 0.5, 0.91, 2, 100]
num_datasets = 100
min_T, max_T = 16, 30
t_uni = torch.tensor([
        1.,  11.,  21.,  31.,  41.,  51.,  61.,  71.,  81.,  91., 101., 111.,
        121., 131., 141., 151., 161., 171., 181., 191., 201., 211., 221., 231.
    ])
unique_intervals = df_h['interval'].unique()
num_unique_intervals = len(unique_intervals)
results_all_alphas = []

for alpha in initial_alpha_values:
    print(f"==== Computing metrics for alpha = {alpha} ====")
    
    model_path = f'pre_trained_models/ablation/PIGPVAE_heating_alpha_{alpha}.pth'
    print(f"Loading model from {model_path}")
    
    kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5))
    GP_inf = GP_inference(kernel)
    
    trainable_alpha_flag = (alpha == 0.91)

    model = PIGPVAE(GP_inf=GP_inf, 
                    hidden_layers_encoder_delta=[10,10], 
                    hidden_layers_encoder_phy=[10], 
                    hidden_layers_decoder_delta=[10,10],
                    activation_decoder_delta=nn.ELU(),
                    initial_alpha=alpha,
                    trainable_alpha=trainable_alpha_flag,
                    mu_prior=torch.tensor(1.7), var_prior=torch.tensor(1)
                   )
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    q_mu_delta_l = []
    q_mu_phy_l   = []
    q_var_phy_l  = []
    
    for x_batch in train_loader:
        with torch.no_grad():
            # Remove extra batch dimension if needed
            x_batch = x_batch.squeeze(0)  # shape: [batch_size, ...]
            
            (
                _,
                _,
                _,
                _,
                _,
                _,
                q_mu_delta_i,
                _,
                q_mu_phy_i,
                q_var_phy_i
            ) = model(x_batch)
            
            q_mu_delta_l.append(q_mu_delta_i)  # shape [batch_size, latent_dim] or similar
            q_mu_phy_l.append(q_mu_phy_i)
            q_var_phy_l.append(q_var_phy_i)
    
    mean_q_mu_phy = sum(q_mu_phy_l) / len(q_mu_phy_l)
    mean_q_var_phy = sum(q_var_phy_l) / len(q_var_phy_l)
    
    q_mu_delta_cat = torch.cat(q_mu_delta_l, dim=1)  # shape [latent_dim, sum_of_batch_sizes]
    mu_s_PIGPVAE = q_mu_delta_cat.mean(dim=1).unsqueeze(1)  # shape [latent_dim, 1]
    
    
    mmd_vals = []
    corr_vals = []
    mdd_vals = []
    
    for i in range(num_datasets):
        min_T, max_T = 11, 30
        torch.manual_seed(0)
        T0 = min_T + (max_T - min_T) * torch.rand(ns)
        difference = 1 + (20 - 1) * torch.rand(ns)
        Ts = T0 + difference
    
        k_values, gd_list, fig, axs = model.generate_and_plot(
            mu_s_PIGPVAE,        # from q_mu_delta
            mean_q_mu_phy,       # from q_mu_phy
            mean_q_var_phy,      # from q_var_phy
            df=df_h,             # or whatever your code references
            num_unique_intervals=num_unique_intervals,
            ns=ns,
            T0=T0,
            Ts=Ts,
            t_uni=t_uni / 240,
            axs_ylim=(0, 35),
            plot=False,
            seed=i
        )
        
        # compute metrics:
        Gd_tensor = torch.stack(gd_list)  
        mmd_val = metrics.compute_mmd(original_data, Gd_tensor)
        corr_val = metrics.correlation_metric(original_data, gd_list)
        mdd_val = metrics.compute_mdd(original_data, gd_list)
        
        mmd_vals.append(mmd_val)
        corr_vals.append(corr_val)
        mdd_vals.append(mdd_val)
    
    mmd_t  = torch.tensor(mmd_vals)
    corr_t = torch.tensor(corr_vals)
    mdd_t  = torch.tensor(mdd_vals)
    
    mmd_mean,  mmd_std  = mmd_t.mean().item(),  mmd_t.std().item()
    corr_mean, corr_std = corr_t.mean().item(), corr_t.std().item()
    mdd_mean,  mdd_std  = mdd_t.mean().item(),  mdd_t.std().item()
    
    print(f"[Alpha = {alpha}] MMD:       mean = {mmd_mean:.4f},  std = {mmd_std:.4f}")
    print(f"[Alpha = {alpha}] Corr:      mean = {corr_mean:.4f}, std = {corr_std:.4f}")
    print(f"[Alpha = {alpha}] MDD:       mean = {mdd_mean:.4f},  std = {mdd_std:.4f}\n")
    
    results_all_alphas.append({
        'alpha': alpha,
        'MMD_mean': mmd_mean,  'MMD_std': mmd_std,
        'Corr_mean': corr_mean,'Corr_std': corr_std,
        'MDD_mean': mdd_mean,  'MDD_std': mdd_std
    })


df_results = pd.DataFrame(results_all_alphas)
print(df_results)


==== Computing metrics for alpha = 0 ====
Loading model from pre_trained_models/ablation/PIGPVAE_heating_alpha_0.pth
[Alpha = 0] MMD:       mean = 0.0800,  std = 0.0046
[Alpha = 0] Corr:      mean = 99.5574, std = 9.4114
[Alpha = 0] MDD:       mean = 0.0314,  std = 0.0027

==== Computing metrics for alpha = 0.1 ====
Loading model from pre_trained_models/ablation/PIGPVAE_heating_alpha_0.1.pth
[Alpha = 0.1] MMD:       mean = 0.0742,  std = 0.0043
[Alpha = 0.1] Corr:      mean = 98.1460, std = 9.1330
[Alpha = 0.1] MDD:       mean = 0.0245,  std = 0.0022

==== Computing metrics for alpha = 0.5 ====
Loading model from pre_trained_models/ablation/PIGPVAE_heating_alpha_0.5.pth
[Alpha = 0.5] MMD:       mean = 0.0700,  std = 0.0038
[Alpha = 0.5] Corr:      mean = 95.6540, std = 7.7288
[Alpha = 0.5] MDD:       mean = 0.0179,  std = 0.0015

==== Computing metrics for alpha = 0.91 ====
Loading model from pre_trained_models/ablation/PIGPVAE_heating_alpha_0.91.pth
[Alpha = 0.91] MMD:       mean = 0.

In [7]:
df_results_formatted = pd.DataFrame({
    'alpha': [f"{row['alpha']:.2f}" for row in results_all_alphas], 
    'MMD':   [f"{row['MMD_mean']:.4f} ({row['MMD_std']:.4f})" for row in results_all_alphas],
    'Corr':  [f"{row['Corr_mean']:.4f} ({row['Corr_std']:.4f})" for row in results_all_alphas],
    'MDD':   [f"{row['MDD_mean']:.4f} ({row['MDD_std']:.4f})" for row in results_all_alphas]
})

print(df_results_formatted)

    alpha              MMD              Corr              MDD
0    0.00  0.0800 (0.0046)  99.5574 (9.4114)  0.0314 (0.0027)
1    0.10  0.0742 (0.0043)  98.1460 (9.1330)  0.0245 (0.0022)
2    0.50  0.0700 (0.0038)  95.6540 (7.7288)  0.0179 (0.0015)
3    0.91  0.0686 (0.0028)  95.6230 (8.3760)  0.0167 (0.0011)
4    2.00  0.0694 (0.0025)  94.5250 (4.7685)  0.0180 (0.0011)
5  100.00  0.0697 (0.0004)  95.7554 (0.0042)  0.0220 (0.0004)


In [8]:
latex_output = df_results_formatted.to_latex(index=False)

# Save the LaTeX table
with open("tables/Metrics_ablation_heating.tex", "w") as f:
    f.write(latex_output)


